# Práctica
> Angel Luis Valdés Sánchez

Utilizando una herramienta o lenguaje de programación, realice lo
siguiente:
1. Descargar un conjunto de datos que contenga datos faltantes del
repositorio UCI.
2. Identificar los datos faltantes y aplicar las ténicas descritas
anteriormente para rellenar la matriz de datos.
3. Elaborar un reporte en una libreta de Jupyter Notebook.

## Descripción del Dataset: Mushrooms (Hongos)

### 📊 **Información General:**
- **Fuente**: UCI Machine Learning Repository (ID: 73)
- **Nombre**: [Mushroom Dataset](https://archive.ics.uci.edu/dataset/73/mushroom)
- **Propósito**: Clasificación de hongos como venenosos (poisonous) o comestibles (edible)
- **Dimensiones**: 8,124 instancias × 23 características

### 🎯 **Variable Target:**
- **Columna**: `poisonous`
- **Valores**: 
  - `'p'` = Poisonous (Venenoso)
  - `'e'` = Edible (Comestible)

### 🍄 **Características del Dataset:**
El dataset contiene **22 características categóricas** que describen aspectos físicos de los hongos, **codificadas con letras**:

**🧢 Características del Sombrero:**
- `cap-shape`: Forma → bell=**b**, conical=**c**, convex=**x**, flat=**f**, knobbed=**k**, sunken=**s**
- `cap-surface`: Superficie → fibrous=**f**, grooves=**g**, scaly=**y**, smooth=**s**
- `cap-color`: Color → brown=**n**, buff=**b**, cinnamon=**c**, gray=**g**, green=**r**, pink=**p**, purple=**u**, red=**e**, white=**w**, yellow=**y**

**🌿 Características de las Láminas (Gill):**
- `gill-attachment`: Unión → attached=**a**, descending=**d**, free=**f**, notched=**n**
- `gill-spacing`: Espaciado → close=**c**, crowded=**w**, distant=**d**
- `gill-size`: Tamaño → broad=**b**, narrow=**n**
- `gill-color`: Color → black=**k**, brown=**n**, buff=**b**, chocolate=**h**, gray=**g**, green=**r**, orange=**o**, pink=**p**, purple=**u**, red=**e**, white=**w**, yellow=**y**

**🌾 Características del Tallo (Stalk):**
- `stalk-shape`: Forma → enlarging=**e**, tapering=**t**
- `stalk-root`: Raíz → bulbous=**b**, club=**c**, cup=**u**, equal=**e**, rhizomorphs=**z**, rooted=**r**, **missing=?** ⚠️
- `stalk-surface-above-ring`: Superficie arriba → fibrous=**f**, scaly=**y**, silky=**k**, smooth=**s**
- `stalk-surface-below-ring`: Superficie abajo → fibrous=**f**, scaly=**y**, silky=**k**, smooth=**s**
- `stalk-color-above-ring`: Color arriba → brown=**n**, buff=**b**, cinnamon=**c**, gray=**g**, orange=**o**, pink=**p**, red=**e**, white=**w**, yellow=**y**
- `stalk-color-below-ring`: Color abajo → brown=**n**, buff=**b**, cinnamon=**c**, gray=**g**, orange=**o**, pink=**p**, red=**e**, white=**w**, yellow=**y**

**🎭 Otras Características:**
- `bruises`: Magulladuras → bruises=**t** (true), no=**f** (false)
- `odor`: Olor → almond=**a**, anise=**l**, creosote=**c**, fishy=**y**, foul=**f**, musty=**m**, none=**n**, pungent=**p**, spicy=**s**
- `veil-type`: Tipo de velo → partial=**p**, universal=**u**
- `veil-color`: Color del velo → brown=**n**, orange=**o**, white=**w**, yellow=**y**
- `ring-number`: Número de anillos → none=**n**, one=**o**, two=**t**
- `ring-type`: Tipo de anillo → cobwebby=**c**, evanescent=**e**, flaring=**f**, large=**l**, none=**n**, pendant=**p**, sheathing=**s**, zone=**z**
- `spore-print-color`: Color esporas → black=**k**, brown=**n**, buff=**b**, chocolate=**h**, green=**r**, orange=**o**, purple=**u**, white=**w**, yellow=**y**
- `population`: Población → abundant=**a**, clustered=**c**, numerous=**n**, scattered=**s**, several=**v**, solitary=**y**
- `habitat`: Hábitat → grasses=**g**, leaves=**l**, meadows=**m**, paths=**p**, urban=**u**, waste=**w**, woods=**d**

### 🔍 **Características Especiales:**
1. **Codificación alfabética**: Cada valor categórico está representado por **una sola letra**
2. **Datos faltantes identificables**: Los valores faltantes aparecen como **'?'** (específicamente en `stalk-root`)

In [2]:
#pip install ucimlrepo

In [3]:
import pandas as pd
from ucimlrepo import fetch_ucirepo 

mushrooms = fetch_ucirepo(id=73) 

X = mushrooms.data.features 
y = mushrooms.data.targets 

data = pd.concat([X, y], axis=1)


In [4]:
print("------------------------")
print("Información del dataset:")
print("------------------------")

print(f"Forma del dataset: {data.shape}")

print("\nPrimeras 5 filas:")
data.head()

------------------------
Información del dataset:
------------------------
Forma del dataset: (8124, 23)

Primeras 5 filas:


,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,poisonous
0,x,s,n,t,p,f,c,n,k,e,...,w,w,p,w,o,p,k,s,u,p
1,x,s,y,t,a,f,c,b,k,e,...,w,w,p,w,o,p,n,n,g,e
2,b,s,w,t,l,f,c,b,n,e,...,w,w,p,w,o,p,n,n,m,e
3,x,y,w,t,p,f,c,n,n,e,...,w,w,p,w,o,p,k,s,u,p
4,x,s,g,f,n,f,w,b,k,t,...,w,w,p,w,o,e,n,a,g,e


## Analizar datos faltantes

In [5]:
# Analizar los datos faltantes por columna en el DataFrame 'data'
missing_counts = data.isnull().sum()
missing_percent = (missing_counts / len(data)) * 100

missing_summary = pd.DataFrame({
    'Columna': data.columns,
    'Valores_Faltantes': missing_counts.values,
    'Porcentaje': missing_percent.values
})

missing_summary

,Columna,Valores_Faltantes,Porcentaje
0,cap-shape,0,0.000000
1,cap-surface,0,0.000000
2,cap-color,0,0.000000
3,bruises,0,0.000000
4,odor,0,0.000000
5,gill-attachment,0,0.000000
6,gill-spacing,0,0.000000
7,gill-size,0,0.000000
8,gill-color,0,0.000000
9,stalk-shape,0,0.000000


## Técnicas de Limpieza de Datos Faltantes

Ahora aplicaremos las diferentes técnicas para manejar los datos faltantes:

1. **Ignorar tuplas con datos faltantes**
2. **Rellenar con constante global**
3. **Rellenar con medidas de tendencia central (media/mediana)**
4. **Rellenar con media/mediana por clase**

### 1. Ignorar tuplas con datos faltantes

In [ ]:
# Técnica 1: Ignorar tuplas con datos faltantes (con análisis inteligente)

print("=== TÉCNICA 1: ELIMINACIÓN DE FILAS CON DATOS FALTANTES ===")
print("Esta técnica elimina filas completas que contienen valores faltantes")

# Analizar la distribución de valores faltantes por fila para tomar decisiones informadas
missing_per_row = data.isnull().sum(axis=1)
missing_percent_per_row = (missing_per_row / data.shape[1]) * 100

print(f"\n--- Análisis de datos faltantes por fila ---")
print(f"Dataset original: {data.shape[0]} filas × {data.shape[1]} columnas")

print(f"\n=== DISTRIBUCIÓN DE FILAS SEGÚN PORCENTAJE DE DATOS FALTANTES ===")
print(f"Filas sin datos faltantes: {(missing_per_row == 0).sum()}")
print(f"Filas con 1-25% de datos faltantes: {((missing_percent_per_row > 0) & (missing_percent_per_row <= 25)).sum()}")
print(f"Filas con > 26% de datos faltantes: {((missing_percent_per_row > 25)).sum()}")

# Mostrar estadísticas detalladas
complete_rows = (missing_per_row == 0).sum()
incomplete_rows = data.shape[0] - complete_rows
complete_percentage = (complete_rows / data.shape[0]) * 100

print(f"\nEstadísticas generales:")
print(f"  • Filas completas: {complete_rows} ({complete_percentage:.1f}%)")
print(f"  • Filas con missing: {incomplete_rows} ({100 - complete_percentage:.1f}%)")

# Estrategia conservadora: Eliminar solo filas con más del 1% de valores faltantes
# En este dataset, esto equivale a eliminar filas que tengan missing en cualquier columna
# ya que cada fila tiene 23 columnas, 1% sería aproximadamente 0.23 columnas
threshold = 0.01
rows_above_threshold = (missing_percent_per_row > threshold * 100).sum()

print(f"\n--- Aplicando umbral del {threshold*100}% ---")
print(f"Filas que superan el umbral: {rows_above_threshold}")

# Aplicar la eliminación
data_drop_threshold = data[missing_percent_per_row <= threshold * 100]

# Calcular estadísticas del resultado
rows_eliminated = data.shape[0] - data_drop_threshold.shape[0]
elimination_percentage = (rows_eliminated / data.shape[0]) * 100
remaining_missing = data_drop_threshold.isnull().sum().sum()

print(f"\n=== RESULTADOS DE LA ELIMINACIÓN ===")
print(f"✅ Filas eliminadas: {rows_eliminated} ({elimination_percentage:.1f}%)")
print(f"✅ Filas restantes: {data_drop_threshold.shape[0]}")
print(f"✅ Valores faltantes restantes: {remaining_missing}")

# Verificación de calidad
if remaining_missing == 0:
    print("🎯 Dataset completamente limpio - Sin valores faltantes")
else:
    print(f"⚠️  Aún quedan {remaining_missing} valores faltantes")

print(f"\n=== IMPACTO DE LA TÉCNICA ===")
print(f"Ventajas: Dataset 100% completo, no introduce valores artificiales")
print(f"Desventajas: Pérdida de {elimination_percentage:.1f}% de información original")


=== ANÁLISIS DE FILAS CON DATOS FALTANTES ===
Filas sin datos faltantes: 5644
Filas con 1-25% de datos faltantes: 2480
Filas con > 26% de datos faltantes: 0

Eliminar filas con > 1% de valores faltantes:
   Filas eliminadas: 2480 (30.5%)
   Filas restantes: 5644
   Valores faltantes restantes: 0


### 2. Rellenar con constante global

In [ ]:
# Técnica 2: Rellenar con constante global (usando valores estándar de Python)

print("=== TÉCNICA 2: RELLENO CON CONSTANTES GLOBALES ===")
print("Esta técnica reemplaza valores faltantes con constantes predefinidas según el tipo de dato")

import numpy as np

# Definir constantes estándar para diferentes tipos de datos
# Estas constantes son valores reconocidos en la industria de datos
UNKNOWN_CATEGORICAL = 'UNK'  # Para variables categóricas - estándar en bases de datos
MISSING_NUMERICAL = -999     # Para variables numéricas enteras - valor centinela clásico
MISSING_FLOAT = np.inf       # Para variables float - infinity, matemáticamente válido
MISSING_BOOLEAN = False      # Para variables booleanas - valor lógico por defecto

print(f"\n--- Constantes definidas ---")
print(f"Variables categóricas: '{UNKNOWN_CATEGORICAL}' (Unknown)")
print(f"Variables enteras: {MISSING_NUMERICAL} (valor centinela)")
print(f"Variables float: {MISSING_FLOAT} (infinito)")
print(f"Variables booleanas: {MISSING_BOOLEAN}")

# Crear copia del dataset para aplicar la técnica
data_global_constant = data.copy()

print(f"\n=== PROCESANDO COLUMNAS ===")
print(f"Analizando {data_global_constant.shape[1]} columnas...")

# Contadores para estadísticas
columns_processed = 0
total_values_filled = 0

for column in data_global_constant.columns:
    original_missing = data_global_constant[column].isnull().sum()
    
    # Solo procesar columnas que tienen valores faltantes
    if original_missing > 0:
        columns_processed += 1
        total_values_filled += original_missing
        
        # Determinar el tipo de dato y aplicar la constante apropiada
        if data_global_constant[column].dtype in ['object', 'category']:
            # Variables categóricas/texto
            data_global_constant[column] = data_global_constant[column].fillna(UNKNOWN_CATEGORICAL)
            print(f"✓ {column} (categórica): {original_missing} valores → '{UNKNOWN_CATEGORICAL}'")
            
        elif data_global_constant[column].dtype == 'bool':
            # Variables booleanas
            data_global_constant[column] = data_global_constant[column].fillna(MISSING_BOOLEAN)
            print(f"✓ {column} (booleana): {original_missing} valores → {MISSING_BOOLEAN}")
            
        elif data_global_constant[column].dtype in ['int64', 'int32', 'int16', 'int8']:
            # Variables enteras
            # Para enteros pequeños usar -999, para grandes convertir a float e usar inf
            if data_global_constant[column].max() < 1000:
                data_global_constant[column] = data_global_constant[column].fillna(MISSING_NUMERICAL)
                print(f"✓ {column} (entero): {original_missing} valores → {MISSING_NUMERICAL}")
            else:
                # Para enteros grandes, convertir a float y usar inf
                data_global_constant[column] = data_global_constant[column].astype(float).fillna(MISSING_FLOAT)
                print(f"✓ {column} (entero→float): {original_missing} valores → {MISSING_FLOAT}")
                
        elif data_global_constant[column].dtype in ['float64', 'float32']:
            # Variables float
            data_global_constant[column] = data_global_constant[column].fillna(MISSING_FLOAT)
            print(f"✓ {column} (float): {original_missing} valores → {MISSING_FLOAT}")
            
        else:
            # Tipo de dato no reconocido, usar valor por defecto
            data_global_constant[column] = data_global_constant[column].fillna(UNKNOWN_CATEGORICAL)
            print(f"? {column} (tipo desconocido): {original_missing} valores → '{UNKNOWN_CATEGORICAL}'")

print(f"\n=== RESULTADOS DEL PROCESAMIENTO ===")
remaining_missing = data_global_constant.isnull().sum().sum()
print(f"✅ Columnas procesadas: {columns_processed}")
print(f"✅ Total de valores rellenados: {total_values_filled}")
print(f"✅ Valores faltantes después del relleno: {remaining_missing}")

# Mostrar estadísticas detalladas de los valores utilizados
print(f"\n=== ESTADÍSTICAS DE VALORES ARTIFICIALES INTRODUCIDOS ===")
artificial_count = 0

for column in data_global_constant.columns:
    if data_global_constant[column].dtype in ['object', 'category']:
        unk_count = (data_global_constant[column] == UNKNOWN_CATEGORICAL).sum()
        if unk_count > 0:
            artificial_count += unk_count
            print(f"  {column}: {unk_count} valores 'UNK' ({unk_count/len(data_global_constant)*100:.1f}%)")
    
    elif data_global_constant[column].dtype in ['float64', 'float32']:
        inf_count = np.isinf(data_global_constant[column]).sum()
        minus999_count = (data_global_constant[column] == MISSING_NUMERICAL).sum()
        if inf_count > 0:
            artificial_count += inf_count
            print(f"  {column}: {inf_count} valores 'inf' ({inf_count/len(data_global_constant)*100:.1f}%)")
        if minus999_count > 0:
            artificial_count += minus999_count
            print(f"  {column}: {minus999_count} valores '-999' ({minus999_count/len(data_global_constant)*100:.1f}%)")
    
    elif data_global_constant[column].dtype in ['int64', 'int32', 'int16', 'int8']:
        minus999_count = (data_global_constant[column] == MISSING_NUMERICAL).sum()
        if minus999_count > 0:
            artificial_count += minus999_count
            print(f"  {column}: {minus999_count} valores '-999' ({minus999_count/len(data_global_constant)*100:.1f}%)")

print(f"\n=== IMPACTO DE LA TÉCNICA ===")
artificial_percentage = (artificial_count / (len(data_global_constant) * data_global_constant.shape[1])) * 100
print(f"Total de valores artificiales: {artificial_count} ({artificial_percentage:.2f}% del dataset)")
print(f"Ventajas: Conserva todas las filas, valores estándar identificables")
print(f"Desventajas: Introduce {artificial_count} valores artificiales que pueden afectar análisis")


=== PROCESANDO COLUMNAS ===
✓ stalk-root (categórica): 2480 valores → 'UNK'

=== RESULTADOS ===
Valores faltantes después de rellenar con constantes globales: 0

=== ESTADÍSTICAS DE VALORES UTILIZADOS ===
  stalk-root: 2480 valores 'UNK' (30.5%)


### 3. Rellenar con medidas de tendencia central

In [16]:
# Técnica 3: Rellenar con medidas de tendencia central (específicamente para stalk-root)

print("=== TÉCNICA 3: RELLENO CON MEDIDAS DE TENDENCIA CENTRAL ===")
print("Esta técnica utiliza estadísticas generales del dataset (sin considerar clases)")

# Crear una copia del dataset original para aplicar la técnica
data_central_tendency = data.copy()

# Enfocarse específicamente en la columna que tiene valores faltantes
col = 'stalk-root'

print(f"\n--- Analizando columna: {col} ---")

# Verificar si la columna tiene valores faltantes
missing_count = data_central_tendency[col].isnull().sum()
total_rows = len(data_central_tendency)
missing_percentage = (missing_count / total_rows) * 100

print(f"Valores faltantes en '{col}': {missing_count} de {total_rows} ({missing_percentage:.2f}%)")

if missing_count > 0:
    print(f"\n--- Aplicando relleno con MODA (valor más frecuente) ---")
    
    # Para variables categóricas, calcular la moda (valor más frecuente)
    # La moda es la medida de tendencia central más apropiada para variables categóricas
    mode_value = data_central_tendency[col].mode()
    
    if len(mode_value) > 0:
        # Mostrar información sobre la moda antes del relleno
        value_counts = data_central_tendency[col].value_counts()
        print(f"Distribución actual de valores en '{col}':")
        print(value_counts.head())
        
        print(f"\nValor de moda seleccionado: '{mode_value[0]}'")
        print(f"Frecuencia de la moda: {value_counts.iloc[0]} apariciones")
        print(f"Porcentaje de la moda: {(value_counts.iloc[0] / (total_rows - missing_count)) * 100:.2f}% de valores válidos")
        
        # Aplicar el relleno
        data_central_tendency[col] = data_central_tendency[col].fillna(mode_value[0])
        
        # Verificar el resultado
        remaining_missing = data_central_tendency[col].isnull().sum()
        filled_count = missing_count - remaining_missing
        
        print(f"\n✅ Resultado del relleno:")
        print(f"  • Valores rellenados: {filled_count}")
        print(f"  • Valores faltantes restantes: {remaining_missing}")
        print(f"  • Éxito del relleno: {(filled_count / missing_count) * 100:.1f}%")
        
    else:
        print(f"⚠️ No se pudo calcular la moda para '{col}' (todos los valores son únicos)")
        
else:
    print(f"✅ La columna '{col}' no tiene valores faltantes")

=== TÉCNICA 3: RELLENO CON MEDIDAS DE TENDENCIA CENTRAL ===
Esta técnica utiliza estadísticas generales del dataset (sin considerar clases)

--- Analizando columna: stalk-root ---
Valores faltantes en 'stalk-root': 2480 de 8124 (30.53%)

--- Aplicando relleno con MODA (valor más frecuente) ---
Distribución actual de valores en 'stalk-root':
stalk-root
b    3776
e    1120
c     556
r     192
Name: count, dtype: int64

Valor de moda seleccionado: 'b'
Frecuencia de la moda: 3776 apariciones
Porcentaje de la moda: 66.90% de valores válidos

✅ Resultado del relleno:
  • Valores rellenados: 2480
  • Valores faltantes restantes: 0
  • Éxito del relleno: 100.0%


### 4. Rellenar con media/mediana por clase

In [ ]:
# Técnica 4: Rellenar con moda por clase (específicamente para stalk-root)

def fill_missing_by_group(dataframe, target_column, columns_to_fill=None, method='mode', fallback_value='UNK'):
    """
    Función reutilizable para rellenar valores faltantes por grupo/clase
    
    Parámetros:
    - dataframe: DataFrame con los datos
    - target_column: Columna para agrupar (variable target)
    - columns_to_fill: Lista de columnas a rellenar (None = todas las columnas con missing)
    - method: 'mode' para categóricas, 'median' para numéricas, 'mean' para promedio
    - fallback_value: Valor de respaldo si no se puede calcular estadística
    
    Retorna:
    - DataFrame con valores faltantes rellenados
    """
    df_filled = dataframe.copy()
    
    # Si no se especifican columnas, usar todas las que tienen valores faltantes
    if columns_to_fill is None:
        columns_to_fill = [col for col in df_filled.columns 
                          if df_filled[col].isnull().sum() > 0 and col != target_column]
    
    results = {}
    
    for column in columns_to_fill:
        if df_filled[column].isnull().sum() == 0:
            continue
            
        print(f"\n--- Procesando columna: {column} ---")
        
        # Mostrar distribución por clase antes del relleno
        missing_by_class = df_filled.groupby(target_column)[column].apply(lambda x: x.isnull().sum())
        print(f"Valores faltantes por clase:\n{missing_by_class}")
        
        if df_filled[column].dtype in ['object', 'category']:
            if method == 'mode':
                # Calcular moda por clase
                def fill_with_mode(group):
                    mode_values = group.mode()
                    if len(mode_values) > 0:
                        return group.fillna(mode_values.iloc[0])
                    else:
                        return group.fillna(fallback_value)
                
                df_filled[column] = df_filled.groupby(target_column)[column].transform(fill_with_mode)
                
                # Mostrar qué valores se usaron por clase
                mode_by_class = df_filled[df_filled[column].notna()].groupby(target_column)[column].agg(lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else fallback_value)
                print(f"Valores de moda utilizados por clase:\n{mode_by_class}")
                
        else:
            # Para variables numéricas
            if method == 'median':
                df_filled[column] = df_filled.groupby(target_column)[column].transform(
                    lambda x: x.fillna(x.median())
                )
            elif method == 'mean':
                df_filled[column] = df_filled.groupby(target_column)[column].transform(
                    lambda x: x.fillna(x.mean())
                )
        
        # Verificar el resultado
        remaining_missing = df_filled[column].isnull().sum()
        results[column] = {
            'original_missing': dataframe[column].isnull().sum(),
            'remaining_missing': remaining_missing,
            'filled_count': dataframe[column].isnull().sum() - remaining_missing
        }
        
        print(f"✅ {column}: {results[column]['filled_count']} valores rellenados, {remaining_missing} restantes")
    
    return df_filled, results

# Aplicar la técnica 4 específicamente a stalk-root
data_class_based = data.copy()
target_column = y.columns[0]

print("=== TÉCNICA 4: RELLENO POR CLASE ===")
print(f"Usando la columna '{target_column}' como variable de agrupación")
print(f"Enfoque específico en la columna: stalk-root")

# Aplicar solo a stalk-root usando nuestra función
data_class_based, fill_results = fill_missing_by_group(
    dataframe=data_class_based,
    target_column=target_column,
    columns_to_fill=['stalk-root'],  # Solo procesar stalk-root
    method='mode',
    fallback_value='UNK'
)

print(f"\n=== RESULTADOS FINALES ===")
print(f"Total de valores faltantes después del relleno: {data_class_based.isnull().sum().sum()}")

# Mostrar estadísticas detalladas del relleno
if 'stalk-root' in fill_results:
    result = fill_results['stalk-root']
    print(f"\nDetalle para stalk-root:")
    print(f"  • Valores faltantes originales: {result['original_missing']}")
    print(f"  • Valores rellenados: {result['filled_count']}")
    print(f"  • Valores faltantes restantes: {result['remaining_missing']}")

# Verificar la distribución final de stalk-root por clase
print(f"\n=== DISTRIBUCIÓN FINAL DE STALK-ROOT POR CLASE ===")
distribution = data_class_based.groupby(target_column)['stalk-root'].value_counts().unstack(fill_value=0)
print(distribution)

=== TÉCNICA 4: RELLENO POR CLASE ===
Usando la columna 'poisonous' como variable de agrupación
Enfoque específico en la columna: stalk-root

--- Procesando columna: stalk-root ---
Valores faltantes por clase:
poisonous
e     720
p    1760
Name: stalk-root, dtype: int64
Valores de moda utilizados por clase:
poisonous
e    b
p    b
Name: stalk-root, dtype: object
✅ stalk-root: 2480 valores rellenados, 0 restantes

=== RESULTADOS FINALES ===
Total de valores faltantes después del relleno: 0

Detalle para stalk-root:
  • Valores faltantes originales: 2480
  • Valores rellenados: 2480
  • Valores faltantes restantes: 0

=== DISTRIBUCIÓN FINAL DE STALK-ROOT POR CLASE ===
stalk-root     b    c    e    r
poisonous                      
e           2640  512  864  192
p           3616   44  256    0


## Conclusiones

### Resumen de Técnicas Aplicadas:

1. **Eliminar filas con datos faltantes**: Perdemos información valiosa pero obtenemos un dataset completamente limpio.

2. **Constante global**: Mantiene todos los datos pero introduce valores artificiales que pueden afectar los análisis.

3. **Tendencia central**: Usa la mediana/moda para rellenar, manteniendo la distribución general de los datos.

4. **Por clase**: Más sofisticado, usa estadísticas específicas por grupo para un relleno más contextual.

5. **Predicción**: La técnica más avanzada, utiliza machine learning para predecir valores basándose en otros atributos.

### Recomendaciones:

- **Para análisis exploratorio**: Usar técnicas 3 o 4 (tendencia central o por clase)
- **Para modelos de machine learning**: Considerar técnica 5 (predicción) o técnica 4 (por clase)
- **Para análisis donde la integridad de datos es crítica**: Técnica 1 (eliminar filas) si la pérdida de datos es aceptable